In [1]:
import pandas as pd
import numpy as np

In [2]:
aznCAN = pd.read_csv("canReport1.csv")

In [4]:
aznCAN=aznCAN.drop(columns = {'Unnamed: 0'})

In [6]:
aznCAN.head(1)

,date,time,type,order id,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35


In [ ]:
# Dropping columns: settlement id, order id, marketplace, fulfillment, and total
# aznCAN = aznCAN.drop(columns = {'settlement id','marketplace','fulfillment','total'})

In [7]:
# I'm having an error with subscripting an iteration because of its type.
# Im going to change the type of the sku column from an object to a string
# so i can manipulate it
aznCAN['sku'] =aznCAN['sku'].astype(str)

In [8]:
# Inserting an extra 'M' into all skus which have the first 3 characters '3M-...'
for i in range(0,len(aznCAN['sku'])):
    if aznCAN['sku'][i][0:3] == '3M-':
        aznCAN['sku'][i] = aznCAN['sku'][i][0:2]+'M'+aznCAN['sku'][i][2:len(aznCAN['sku'][i])]

In [10]:
aznCAN.head(1)

,date,time,type,order id,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35


In [11]:
# In this cell, i wil be adding the main IDs first and then landing costs to each item by importing the 'databaseVersion1' 
databaseLandingCosts = pd.read_csv('azmDatabaseVersion1.csv')

In [12]:
databaseLandingCosts.head(1)

,ID Group,Sub-ID,landing cost
0,3MM-1527-1,3MM-1527-1,4.73


In [14]:
# aznCAN = aznCAN.drop(columns = {'Unnamed: 0'})

In [ ]:
# aznCAN.head(1)

In [15]:
# Here I am inserting a new column in between 'type' and 'sku' to include the main SKU ids which each SKU belongs too
aznCAN.insert(4,'group-sku', '')

In [17]:
aznCAN.head(1)

,date,time,type,order id,group-sku,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35


In [18]:
# Inserting the group-skus 
aznCAN['sku'] = aznCAN['sku'].str.upper() 
for i in range(0,len(aznCAN['sku'].unique())):
    for j in range(0,len(databaseLandingCosts['Sub-ID'].unique())):
            if aznCAN['sku'].unique()[i] == databaseLandingCosts['Sub-ID'][j]:
                aznCAN.loc[aznCAN['sku']== aznCAN['sku'].unique()[i],'group-sku'] = databaseLandingCosts['ID Group'][j]

                

In [19]:
# Adding in the landing costs
aznCAN['landing cost'] = '0'

for i in range(0,len(aznCAN['sku'].unique())):
    for j in range(0,len(databaseLandingCosts['Sub-ID'])):
        if aznCAN['sku'].unique()[i] == databaseLandingCosts['Sub-ID'][j]:
            aznCAN.loc[aznCAN['sku']==aznCAN['sku'].unique()[i],'landing cost'] = databaseLandingCosts['landing cost'][j]

In [20]:
aznCAN.head()

,date,time,type,order id,group-sku,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees,landing cost
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35,27.4
1,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,-20.96,-17.24,27.4
2,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,0.00,-7.35,27.4
3,2019-01-01,7:58:37 PM PST,Order,702-6870704-8572226,NaN,LUX-NEO110B,Luxe Bidet Neo 110 - Fresh Water Non-Electric ...,1.0,Brampton,Ontario,L6P 1H2,33.95,-5.09,-7.35,14.65
4,2019-01-01,7:58:43 PM PST,Order,701-4182545-1251421,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Richmond Hill,Ontario,L4C 9L5,69.85,-10.48,-7.35,27.4


In [21]:
aznCAN.dtypes
aznCAN['landing cost'] = pd.to_numeric(aznCAN['landing cost'])

In [22]:
#### In this cell i am adjusting the landing costs with respect to the amount of quantity bought from that record
for i in range(0,len(aznCAN['sku'])):
    aznCAN['landing cost'][i] = aznCAN['landing cost'][i]*aznCAN['quantity'][i]

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,time,type,order id,group-sku,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees,landing cost
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35,27.40
1,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,-20.96,-17.24,27.40
2,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,0.00,-7.35,27.40
3,2019-01-01,7:58:37 PM PST,Order,702-6870704-8572226,NaN,LUX-NEO110B,Luxe Bidet Neo 110 - Fresh Water Non-Electric ...,1.0,Brampton,Ontario,L6P 1H2,33.95,-5.09,-7.35,14.65
4,2019-01-01,7:58:43 PM PST,Order,701-4182545-1251421,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Richmond Hill,Ontario,L4C 9L5,69.85,-10.48,-7.35,27.40
5,2019-01-01,8:09:01 PM PST,Order,702-4449880-3123447,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Montreal,Quebec,H1Y 1N9,69.85,-10.48,-7.35,27.40
6,2019-01-01,8:09:10 PM PST,Order,701-7922138-7999447,,PRK-12-02(1),Spectra PAR12-02 Parker Laboratories 360 Elect...,1.0,Greenfield Park,Quebec,J4V 3J4,8.33,-1.00,-6.58,0.00
7,2019-01-01,10:47:16 PM PST,Order,702-2270298-4585813,,PRK-12-02(1),Spectra PAR12-02 Parker Laboratories 360 Elect...,1.0,London,Ontario,N6E 2M9,8.33,-1.00,-6.58,0.00
8,2019-01-01,10:49:32 PM PST,Order,702-0166729-9569868,NaN,LUX-NEO185B,Luxe Bidet Neo 185 (Elite Series) - Self Clean...,1.0,London,Ontario,N6P 1B3,49.95,-7.49,-14.89,24.03
9,2019-01-01,11:01:00 PM PST,Order,702-5598472-3165837,NaN,LUX-NEO185B,Luxe Bidet Neo 185 (Elite Series) - Self Clean...,1.0,Trois-Rivieres,Quebec,G8Z 1X5,49.95,-7.49,-15.73,24.03


In [23]:
aznCAN

,date,time,type,order id,group-sku,sku,description,quantity,order city,order state,order postal,product sales,selling fees,fba fees,landing cost
0,2019-01-01,7:56:01 PM PST,Order,702-5889472-3075408,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Maple,Ontario,L6A 0H6,69.85,-10.48,-7.35,27.40
1,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,-20.96,-17.24,27.40
2,2019-01-01,7:56:08 PM PST,Order,701-7218821-3163451,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Toronto,Ontario,M2J 0A9,69.85,0.00,-7.35,27.40
3,2019-01-01,7:58:37 PM PST,Order,702-6870704-8572226,NaN,LUX-NEO110B,Luxe Bidet Neo 110 - Fresh Water Non-Electric ...,1.0,Brampton,Ontario,L6P 1H2,33.95,-5.09,-7.35,14.65
4,2019-01-01,7:58:43 PM PST,Order,701-4182545-1251421,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Richmond Hill,Ontario,L4C 9L5,69.85,-10.48,-7.35,27.40
5,2019-01-01,8:09:01 PM PST,Order,702-4449880-3123447,NaN,LUX-NEO320B,Luxe Bidet Neo 320 (Elite Series) - Self Clean...,1.0,Montreal,Quebec,H1Y 1N9,69.85,-10.48,-7.35,27.40
6,2019-01-01,8:09:10 PM PST,Order,701-7922138-7999447,,PRK-12-02(1),Spectra PAR12-02 Parker Laboratories 360 Elect...,1.0,Greenfield Park,Quebec,J4V 3J4,8.33,-1.00,-6.58,0.00
7,2019-01-01,10:47:16 PM PST,Order,702-2270298-4585813,,PRK-12-02(1),Spectra PAR12-02 Parker Laboratories 360 Elect...,1.0,London,Ontario,N6E 2M9,8.33,-1.00,-6.58,0.00
8,2019-01-01,10:49:32 PM PST,Order,702-0166729-9569868,NaN,LUX-NEO185B,Luxe Bidet Neo 185 (Elite Series) - Self Clean...,1.0,London,Ontario,N6P 1B3,49.95,-7.49,-14.89,24.03
9,2019-01-01,11:01:00 PM PST,Order,702-5598472-3165837,NaN,LUX-NEO185B,Luxe Bidet Neo 185 (Elite Series) - Self Clean...,1.0,Trois-Rivieres,Quebec,G8Z 1X5,49.95,-7.49,-15.73,24.03


In [24]:
# exporting the file to a csv
aznCAN.to_csv(r'canReportPreAgg.csv')

# Creating the Aggregate report, make sure to delete unnecessary columns on excel or here (i did it on excel)

In [ ]:
aznCAN = pd.read_csv("canReportPReAgg.csv")

In [ ]:
aznCAN['selling fees'] = -(aznCAN['selling fees'])
aznCAN['fba fees'] = -(aznCAN['fba fees'])

In [ ]:
# Extracting all the type Order records. This is because these are the actual transactions which have gone through
aznCANOrders = aznCAN.loc[aznCAN['type']=="Order"]

In [ ]:
aznCANOrders

In [ ]:
aznCANOrders['sku'].unique()
aznCanAgg = []
for i in aznCANOrders['sku'].unique():
     aznCanAgg.append(i)

In [ ]:
aznCanAgg = pd.DataFrame(aznCanAgg)

In [ ]:
aznCanAgg = aznCanAgg.rename(columns = {0:"sku"})

In [ ]:
aznCanAgg['Total-Quantity-Sold'] = ''
aznCanAgg['Total-Product-Revenue'] = ''
aznCanAgg['Total-Selling-Fee'] = ''
aznCanAgg['Total-Fba-Fees'] = ''
aznCanAgg['Total-Landing-Costs'] = ''
aznCanAgg['Total-Product-Profit'] = ''
aznCanAgg['R.O.I'] = ''
aznCanAgg['Profit Per Quantity Sold'] = ''


In [ ]:
# Sorting and reseting indexes and dropping

#aznCanAgg = aznCanAgg.sort_values(by ='sku', ascending = True).reset_index()
#aznCanAgg = aznCanAgg.drop(columns = ['index','level_0'])

In [ ]:
aznCANOrders = aznCANOrders.groupby('sku').agg({'quantity':'sum','product sales':'sum','selling fees':'sum','fba fees':'sum','landing cost':'sum'}).reset_index()

In [ ]:
for i in range(0,len(aznCanAgg['sku'])):
    for j in range(0,len(aznCANOrders['sku'])):
        if  aznCanAgg['sku'][i] == aznCANOrders['sku'][j]:
            aznCanAgg['Total-Quantity-Sold'][i] = aznCANOrders['quantity'][j]
            aznCanAgg['Total-Product-Revenue'][i] = aznCANOrders['product sales'][j]
            aznCanAgg['Total-Selling-Fee'][i] = aznCANOrders['selling fees'][j]
            aznCanAgg['Total-Fba-Fees'][i] = aznCANOrders['fba fees'][j]
            aznCanAgg['Total-Landing-Costs'][i] = aznCANOrders['landing cost'][j]
            aznCanAgg['Total-Product-Profit'][i] = ''
            aznCanAgg['R.O.I'][i] = ''
            aznCanAgg['Profit Per Quantity Sold'][i] = ''

In [ ]:
# Calculating the profit for that item 
aznCanAgg['Total-Product-Profit'] = aznCanAgg['Total-Product-Revenue'] - (aznCanAgg['Total-Selling-Fee'] +aznCanAgg['Total-Fba-Fees']+aznCanAgg['Total-Landing-Costs'])

In [ ]:
# claculating the profit per quantity sold 
aznCanAgg['Profit Per Quantity Sold'] = aznCanAgg['Total-Product-Profit']/aznCanAgg['Total-Quantity-Sold']

In [ ]:
aznCanAgg['R.O.I'] = aznCanAgg['Total-Product-Profit']/aznCanAgg['Total-Quantity-Sold']

In [ ]:
aznCanAgg = aznCanAgg.rename(columns = {"R.O.I":"Return on Sales"})

In [ ]:
aznCanAgg

In [ ]:
aznCanAgg.to_csv(r"aznCanAggregateReport.csv")

# Splitting up the pre aggregate report into their types, orders, refunds... 

In [26]:
import pandas as pd
aznCAN = pd.read_csv("canReportPreAgg.csv")

FileNotFoundError: [Errno 2] File b'canReportPreAgg.csv' does not exist: b'canReportPreAgg.csv'

In [ ]:
aznCAN['selling fees'] = -(aznCAN['selling fees'])
aznCAN['fba fees'] = -(aznCAN['fba fees'])

In [ ]:
aznCAN['type'].unique()

In [ ]:
# Orders
orders = aznCAN.loc[aznCAN['type']=='Order'].reset_index()
refund = aznCAN.loc[aznCAN['type']=='Refund'].reset_index()
fbaInventoryFee = aznCAN.loc[aznCAN['type']=='FBA Inventory Fee'].reset_index()
adjustments = aznCAN.loc[aznCAN['type']=='Adjustment'].reset_index()
transfer = aznCAN.loc[aznCAN['type']=='Transfer'].reset_index()
serviceFee = aznCAN.loc[aznCAN['type']=='Service Fee'].reset_index()
orderRetroCharge = aznCAN.loc[aznCAN['type']=='Order_Retrocharge'].reset_index()

In [ ]:
orders = orders.drop(columns = {'index'})
refund = refund.drop(columns = {'index'})
fbaInventoryFee= fbaInventoryFee.drop(columns = {'index'})
adjustments = adjustments.drop(columns = {'index'})
transfer = transfer.drop(columns = {'index'})
serviceFee=serviceFee.drop(columns = {'index'})
orderRetroCharge=orderRetroCharge.drop(columns = {'index'})

In [ ]:
print(f"orders = {len(orders)}")
print(f"refund = {len(refund)}")
print(f"fbaInventoryfee = {len(fbaInventoryFee)}")
print(f"adjustments = {len(adjustments)}")
print(f"transfer = {len(transfer)}")
print(f"serviceFee = {len(serviceFee)}")
print(f"orderRetroCharge = {len(orderRetroCharge)}")


In [ ]:
orders.to_csv('CANorders.csv')
refund.to_csv('CANrefund.csv')
fbaInventoryFee.to_csv('CANfbaInventoryFee.csv')
adjustments.to_csv('CANadjustments.csv')
transfer.to_csv('CANtransfer.csv')
serviceFee.to_csv('CANserviceFee.csv')
orderRetroCharge.to_csv('CANorderRetroCharge.csv')